In [36]:
import os

# ユーザーのディレクトリを作成する関数
def create_user_directory(data):

    user_id = data["user_id"]

    base_path = "../user"
    user_path = os.path.join(base_path, user_id)
    
    # ユーザーディレクトリを作成（すでに存在する場合は何もしない）
    try:
        os.makedirs(user_path, exist_ok=True)
        os.makedirs(os.path.join(user_path, "CartPole"), exist_ok=True)
        os.makedirs(os.path.join(user_path, "FlappyBird"), exist_ok=True)
        return {"message": "successfully"}
    except Exception as e:
        return {"message": str(e)}

# 使用例
data = {
    "user_id": "example_user_id"
}

response = create_user_directory(data)
print(response)

{'message': 'successfully'}


In [37]:
import os

# プロジェクトディレクトリを作成する関数
def create_project_directory(data):

    user_id = data["user_id"]
    project_name = data["project_name"]

    base_path = "../user"
    user_path = os.path.join(base_path, user_id)
    project_path = os.path.join(user_path, project_name)
    
    try:
        os.makedirs(project_path, exist_ok=True)
        return {"message": "successfully"}
    except Exception as e:
        return {"message": str(e)}

# 使用例
data = {
    "user_id": "example_user_id",
    "project_name": "example_project"
}
response = create_project_directory(data)
print(response)


{'message': 'successfully'}


In [38]:
import os

def create_model_directory_from_dict(data):

    user_id = data["user_id"]
    project_name = data["project_name"]
    model_id = data["model_id"]

    base_path = "../user"
    project_path = os.path.join(base_path, user_id, project_name)
    model_path = os.path.join(project_path, model_id)
    
    try:
        os.makedirs(model_path, exist_ok=True)
        return {"message": "successfully"}
    except Exception as e:
        return {"message": str(e)}

# 使用例
data = {
    "user_id": "example_user_id",
    "project_name": "example_project",
    "model_id": "example_model"
}
response = create_model_directory_from_dict(data)
print(response)


{'message': 'successfully'}


In [ ]:
def make_conv_layer(layer):
    return f'            nn.Conv2d({layer["in_channels"]}, {layer["out_channel"]}, {layer["kernel_size"]}, stride={layer["stride"]}, padding={layer["padding"]}),\n'

def make_pool_layer(pool_type, kernel_size, stride, padding):
    return f'            nn.{pool_type}({kernel_size}, stride={stride}, padding={padding}),\n'

def make_dropout_layer(dropout_p):
    return f'            nn.Dropout(p={dropout_p}),\n'

def make_batchnorm_layer(num_features):
    return f'            nn.BatchNorm2d({num_features}),\n'

def make_linear(input_size, output_size):
    return f'            nn.Linear({input_size}, {output_size}),\n'

def make_activ(activ_name):
    return f'            nn.{activ_name}(),\n'

def make_flatten_bif(flutten_bif):
    return f'            nn.{flutten_bif}(1),\n'

def make_python_code(data):
    py_1 = '''
import torch.nn as nn
'''
    py_2 = '''
class Simple_NN(nn.Module):
    def __init__(self):
        super(Simple_NN, self).__init__()
'''
    py_3 = '''
        self.seq = nn.Sequential(
'''
    py_4 = '''
            )
'''
    py_5 = '''
    def forward(self, x):
        return self.seq(x)
'''
    structure = data.get('structure')
    input_layer = structure.get('InputLayer')
    conv_layers = structure.get('ConvLayer')
    middle_layers = structure.get('MiddleLayer')
    output_size = structure.get('OutputLayer')
    flutten_way = structure.get('Flatten_way')
    save_dir = f'../user/{data.get("user_id")}/{data.get("Project_name")}/{data.get("model_id")}'

    in_channels = input_layer[2]
    for layer in conv_layers:
        layer_type = layer.get('layer_type')
        if layer_type == 'conv2d':
            py_3 += make_conv_layer(layer)
            py_3 += make_activ(layer.get('activ_func'))
            in_channels = layer.get('out_channel')
        elif layer_type == 'pool':
            py_3 += make_pool_layer(layer.get('pool_type'), layer.get('kernel_size'), layer.get('stride'), layer.get('padding'))
        elif layer_type == 'dropout':
            py_3 += make_dropout_layer(layer.get('dropout_p'))
        elif layer_type == 'batchnorm':
            py_3 += make_batchnorm_layer(in_channels)

    # Global Poolingの追加
    if flutten_way == 'GAP':
        py_3 += '            nn.AdaptiveAvgPool2d(1),\n'
    elif flutten_way == 'GMP':
        py_3 += '            nn.AdaptiveMaxPool2d(1),\n'

    # Flatten層の追加
    py_3 += '            nn.Flatten(),\n'

    input_size = middle_layers[0].get('input_size')
    for layer in middle_layers:
        py_3 += make_linear(input_size, layer.get('input_size'))
        py_3 += make_activ(layer.get('activ_func'))
        input_size = layer.get('input_size')

    py_3 += make_linear(input_size, output_size)

    python_code = py_1 + py_2 + py_3 + py_4 + py_5
    file_name = 'model_config.py'

    try:
        with open(f"{save_dir}/{file_name}", 'w') as file:
            file.write(python_code)
        return {"message": "successfully"}
    except Exception as e:
        return {"message": str(e)}

# 使用例
data = {
    "model_type": "Conv",
    "user_id": "example_user_id",
    "Project_name": "example_project",
    "model_id": "example_model",
    "structure": {
        "InputLayer": [28, 28, 1],
        "Pretreatment": "none",
        "ConvLayer": [
            {
                "layer_type": "conv2d",
                "in_channels": 1,
                "out_channel": 64,
                "kernel_size": 3,
                "stride": 1,
                "padding": 0,
                "activ_func": "ReLU"
            },
            {
                "layer_type": "padding",
                "kernel_size": 3,
                "stride": 1,
                "padding": 0
            },
            {
                "layer_type": "dropout",
                "dropout_p": 0.2
            },
            {
                "layer_type": "batchnorm"
            }
        ],
        "Flatten_way": "GMP",
        "MiddleLayer": [
            {
                "input_size": 100,
                "activ_func": "Tanh"
            },
            {
                "input_size": 100,
                "activ_func": "Sigmoid"
            }
        ],
        "OutputLayer": 10
    }
}
response = make_python_code(data)
print(response)
